# NLP TD 2 LAPASSAT Louis

Link of the [GitHub (course)](https://github.com/edupoux/MVA_2020_SL/tree/master/TD_%232).

**The goal of this assignment is to develop a basic probabilistic parser for French that is based on the CYK algorithm and the PCFG model and that is robust to unknown words.**

The goal of this assignment is not to produce high-accuracy parsers. Its goal is rather for you to build a working statistical constituency parsing architecture based on the CYK algorithm, with a module for handling out-of-vocabulary words (OOVs) based on edit distance and word embedding similarity.

In [1]:
# !pip install PYEVALB # install evalb

In [2]:
### Here import the libraries
import pickle
import numpy as np
from random import shuffle, seed
from nltk import Tree, Nonterminal, induce_pcfg
import re
from pprint import pprint
import operator
import tqdm as tqdm
from difflib import ndiff
from PYEVALB import scorer
from PYEVALB import parser

# Dataset

We use the SEQUOIA treebank v6.0 (file in the GitHub, bracketed format):

 * split it into 3 parts (80% / 10% / 10%).
 * use the 80% for training (extract CFG rules + learn CFG rule probabilities)
 * use the first 10% for development purposes (whatever you want to use it)
 * use the last 10% to evaluate your parser. To keep it simple, you can evaluate your part-of-speech accuracy only, i.e. via the percentage of tokens for which your parser choses the correct part-of-speech. For your information, a standard tool for constituency parse evaluation is [evalb](https://nlp.cs.nyu.edu/evalb/)


In [3]:
def load_data(path):
    ### load the data
    with open(path, "r", encoding='utf-8') as file: # open the file
        content = file.readlines() # read line by line
    content = [x.strip() for x in content]  # remove the '\n' at the end of each line + ( ), [2:-1]
    
    ### now we drop functional labels
    new_content = []
    for sentence in content: # loop over all the sentence
        matches = re.findall('-.+? ', sentence) # find all text starting from '-' and ending with ' '
        matches = [match for match in matches if match[-2] != ')'] # remove false functional labels
        for match in matches: # loop over the matches
            sentence = re.sub(match, ' ', sentence) # remove the match text from the sentence
        new_content.append(sentence) # save the treated sentence
    return new_content

def transform_to_sentence(data):
    new_data = []
    for sentence in data:
        new_data.append(' '.join(Tree.fromstring(sentence).leaves()))
    return new_data


### get data
content = load_data('sequoia-corpus+fct.txt')

### split the data
seed(42) # set the seed for random shuffle
# shuffle(content) # first shuffle the data
train_indice = int(len(content) * 0.8) # set the indice for the training set
test_indice = int(len(content) * 0.1) + train_indice # set the indice for the testing set
train, test, evaluation = content[:train_indice], content[train_indice:test_indice], content[test_indice:]

### transform the data into sentence datasets
train_sentences = transform_to_sentence(train)
test_sentences = transform_to_sentence(test)
evaluation_sentences = transform_to_sentence(evaluation)

# Module to extract a PCFG

module to extract a PCFG from the training corpus provided (see below), made of:

 * a probabilistic context-free grammar whose terminals are part-of-speech tags
 * a probabilistic lexicon, i.e. triples of the form (token, part-of-speech tag, probability) such that the sum of the probabilities for all triples for a given token sums to 1.


In [4]:
def get_pcfg(dataset):
    """Extract the probabilistic context-free grammar from a dataset"""
    productions = []
    ### loop over trees (first convert bracketed string into tree)
    for tree in [Tree.fromstring(tree, remove_empty_top_bracketing=True) for tree in train]:
        tree.collapse_unary(collapsePOS=True)
        tree.chomsky_normal_form(horzMarkov=1) # chomsky_normal_form(horzMarkov=2) # um_chomsky_normal_form() 
        productions += tree.productions() # add productions
    starting_state = Nonterminal('SENT') # starting state, use tree.label() to find 'SENT'
    grammar = induce_pcfg(starting_state, productions) # induce PCFG
    return grammar

def get_lexicon(grammar):
    """Extract probabilistic lexicon"""
    dic = {}
    for e in grammar.productions():
        token = e.rhs()
        part_of_speech_tag = e.lhs()
        probability = e.prob()
        if isinstance(part_of_speech_tag, Nonterminal):
            if isinstance(token, tuple):
                if isinstance(token[0], str):
                    if token[0] in dic:
                        if part_of_speech_tag in dic[token[0]]:
                            print('nan mais allo quoi, error')
                        else:
                            dic_temp = dic[token[0]]
                            dic[token[0]] = {part_of_speech_tag: probability, **dic_temp}
                    else:
                        dic = {token[0]: {part_of_speech_tag: probability}, **dic}
            elif isinstance(token, str):
                print('allo quoi, error')
    
    ### normalize per token
    for token in dic:
        total = sum(dic[token].values())
        for key in dic[token]:
            dic[token][key] /= total
    return dic

grammar = get_pcfg(train) # infer the grammer from the train 
lexicon = get_lexicon(grammar) # infer the lexicon

In [5]:
print(f'There is {len(grammar.productions())} productions in your grammar. Below is a quick look at some of them: \n')
pprint(grammar.productions()[:10])

There is 12706 productions in your grammar. Below is a quick look at some of them: 

[SENT -> NP+NPP [0.00242033],
 NP+NPP -> 'Gutenberg' [0.00247525],
 SENT -> NP SENT|<VN> [0.137959],
 NP -> DET NC [0.220609],
 DET -> 'Cette' [0.00236469],
 NC -> 'exposition' [0.00120797],
 SENT|<VN> -> VN SENT|<Ssub> [0.0439952],
 VN -> CLO V [0.0189378],
 CLO -> 'nous' [0.06],
 V -> 'apprend' [0.000483325]]


In [6]:
print(f'There is {len(lexicon)} keys in your lexicon. Here is a quick look at him:\n')
dict(list(lexicon.items())[:10])

There is 8958 keys in your lexicon. Here is a quick look at him:



{'prouvé': {VPP: 1.0},
 'affirmait': {VN+V: 1.0},
 'daté': {VPP: 1.0},
 'valider': {VN+VINF: 1.0},
 'filières': {NC: 1.0},
 'complémentarité': {NC: 1.0},
 'Soulignant': {VN+VPR: 1.0},
 'Tiananmen': {NP+NPP: 1.0},
 'rencontrés': {VPP: 1.0},
 'au_cours_du': {P+D: 1.0}}

# OOV module

OOV module that assigns a (unique) part-of-speech to any token not included in the lexicon extracted from the training corpus. The underlying idea is to assign to an OOV the part-of-speech of a "similar" word. This similarity will be computed as a combination of formal similarity (to handle spelling errors) and embedding similarity (as measured by cosine similarity, i.e. scalar product between normalised vectors), to handle both spelling errors and genuine unknown words; you must design a reasonable way to combine these two similarities. For embedding similarity, you will use [the Polyglot embedding lexicon for French](https://sites.google.com/site/rmyeid/projects/polyglot) (see the [tutorial on Polyglot embeddings](https://nbviewer.jupyter.org/gist/aboSamoor/6046170); you can re-use this code)

In [7]:
from numba import jit

class OOV():
    def __init__(self, filepath, lexicon):
        ### load words + embeddings
        self.words, self.embeddings = pickle.load(open(filepath, 'rb'), encoding='latin1')
        self.embeddings_norm = [np.linalg.norm(norm) for norm in self.embeddings]
        self.word2id = {word: i for i, word in enumerate(self.words)}
        print(f"There is {len(self.words)} unique words and embddings shape is {self.embeddings.shape}.")
        
        ### save lexicon
        self.lexicon = lexicon

        ### add a list to store new vocabulary
        self.learned_vocabulary = {}
        
    def assign_part_of_speech_unknow(self, unknown_word):

        if unknown_word in self.learned_vocabulary:
            return self.learned_vocabulary[unknown_word]

        ### get similar words
        max_words = 20000
        most_similar_words = self.most_similar_words(unknown_word, k=max_words)
        
        ### loop over all similar words and try to get a match with lexicon
        i = 0
        similar_word = most_similar_words[i]
        while similar_word not in self.lexicon and i < max_words:
            similar_word = most_similar_words[i]
            i += 1
        
        ### if we didn't find anything return error
        if i >= max_words:
            print("Couldn't find anything correct!")
            return None
        
        ### get the most probable nonterminal
        pred_nonterminal, probability = max(lexicon[similar_word].items(), key=operator.itemgetter(1))

        ### store the result
        self.learned_vocabulary[unknown_word] = (pred_nonterminal, probability)
        
        return pred_nonterminal, probability
        
    def cosine_similarity(self, word1, word2):
        ### Return the cosine similarity
        vec_word_1 = self.embeddings[self.word2id[word1]]
        vec_word_2 = self.embeddings[self.word2id[word2]]
        vec_word_1_norm = self.embeddings_norm[self.word2id[word1]]
        vec_word_2_norm = self.embeddings_norm[self.word2id[word2]]
        return np.dot(vec_word_1, vec_word_2) / (vec_word_1_norm * vec_word_2_norm)
    
    def levenshtein_distance(self, str1, str2):
        counter = {"+": 0, "-": 0}
        distance = 0
        for edit_code, *_ in ndiff(str1, str2): # just take - or + (the code)
            if edit_code == " ":
                distance += max(counter.values())
                counter = {"+": 0, "-": 0}
            else: 
                counter[edit_code] += 1
        distance += max(counter.values())
        return distance
    
    def damereau_levenshtein_distance(self, s1, s2):
        len_s1 = len(s1)
        len_s2 = len(s2)
        m = np.zeros((len_s1, len_s2), dtype=int)
        m[:, 0] = range(len_s1)
        m[0, :] = range(len_s2)
        for i in range(1, len_s1):
            for j in range(1, len_s2):
                if s1[i] == s2[j]:
                    m[i, j] = min(m[i - 1, j], m[i, j - 1], m[i - 1, j - 1] - 1) + 1
                else:
                    m[i, j] = min(m[i - 1, j], m[i, j - 1], m[i - 1, j - 1]) + 1
        return m[-1, -1]
    
    def most_similar_words(self, unknown_word, damereau=True, speed=True, k=5):
        ### compute the score
        if unknown_word in self.words:
            res_cos = [self.cosine_similarity(unknown_word, word) for word in self.words]
        else:
            res_cos = [0 for word in self.words]
        if damereau:
            if speed and unknown_word in self.words:
                upper_limit = np.quantile(res_cos, 0.8)
                res_lev = []
                for i, word in enumerate(self.words):
                    if res_cos[i] < upper_limit:
                        res_lev.append(100)
                    else:
                        res_lev.append(self.damereau_levenshtein_distance(unknown_word, word))
            else:
                res_lev = [self.damereau_levenshtein_distance(unknown_word, word) for word in self.words]
        else:
            res_lev = [self.levenshtein_distance(unknown_word, word) for word in self.words]
        
        ### scale in -1 to 1
        res_lev = - np.array(res_lev) # - because if lev dist is big it is bad
        res_lev = 2 * ((res_lev - res_lev.min()) / (res_lev.max() - res_lev.min())) - 1 # to [-1, 1]
        
        ### average 
        scores = 0.5 * res_lev + 0.5 * np.array(res_cos)
        
        ### get top k words
        indice_top_k = np.argsort(scores)[::-1][1:k + 1]
        
        return [self.words[i] for i in indice_top_k]

In [8]:
oov = OOV('polyglot-fr.pkl', lexicon)

There is 100004 unique words and embddings shape is (100004, 64).


In [9]:
oov.most_similar_words('tres')

['trés', 'trop', 'très', 'assez', 'infiniment']

In [10]:
oov.most_similar_words('etre')

['être', "m'être", 'estre', 'ete', "qu'être"]

In [11]:
oov.assign_part_of_speech_unknow('etre')

(VINF, 0.9318137094063546)

In [12]:
oov.assign_part_of_speech_unknow('tres')

(ADV, 1.0)

# CYK algorithm

probabilistic implementation of the CYK algorithm that takes tokenised sentences as an input. In other words, the input of the parser are files with one sentence per line, and each sentence is formed of tokens separated from one another by whitespace characters. The output should be in the same bracketed format as the training data. Regarding the probabilistic part, you will adapt the CYK algorithm so that only the best (i.e. most probable) way to rewrite an instanciated non-terminal symbol is retained. This will give you a recursive, straigtforward way to then retrieve the best (i.e. most probable) parse tree for the whole sentence.

In [13]:
def inverse_lexicon(lexicon):
    ### part_of_speech: {word: proba, etc.}
    dic = {}
    for key in lexicon:
        for X in lexicon[key]:
            if X in dic:
                dic[X] = {key: lexicon[key][X], **dic[X]}
            else:
                dic[X] = {key: lexicon[key][X]}
    return dic

def get_rules(grammar):
    is_nonter = lambda x: isinstance(x[0], Nonterminal) and isinstance(x[1], Nonterminal)
    dic, dic_proba = {}, {}
    for production in grammar.productions():
        if production.lhs() in dic:
            if len(production.rhs()) > 1:
                if is_nonter(production.rhs()):
                    dic[production.lhs()].append(production.rhs())
                    dic_proba[production.lhs()] = {production.rhs(): production.prob(), **dic_proba[production.lhs()]}
        else:
            if len(production.rhs()) > 1:
                if is_nonter(production.rhs()):
                    dic[production.lhs()] = [production.rhs()]
                    dic_proba[production.lhs()] = {production.rhs(): production.prob()}
    return dic, dic_proba

def get_tree(ch, bp, i, j, X):
    if i == j:
          return "".join([str(X),' ', ch[i]])
    else:
        try:
            Y, Z, s = bp[i, j, X]
            return "".join([str(X),' (', get_tree(ch, bp, i, s, Y),') (', get_tree(ch, bp, s+1, j, Z),')'])
        except:
            return "".join([str(X),' ', ch[i]])

In [14]:
def cyk(sentence, lexicon, inversed_lexicon, NonTerminals, rules, rules_prob):
    n = len(sentence)
    pi, bp = {}, {}

    ### Init
    for i in range(n):
        if sentence[i] in lexicon: # check if we already see the word
            terminal = sentence[i]
            for X in inversed_lexicon: # get probability          
                if X in lexicon[terminal]: 
                    pi[i, i, X] = lexicon[terminal][X]
                else:
                    pi[i, i, X] = 0
        else:
            terminal, proba_terminal = oov.assign_part_of_speech_unknow(sentence[i]) # if unknown get a similar pos
            for X in inversed_lexicon: # get probability          
                if X == terminal: 
                    pi[i, i, X] = proba_terminal
                else:
                    pi[i, i, X] = 0

    ## main loop
    for l in range(n - 1):
        for i in range(n - l - 1): 
            j = i + l + 1
            for X in rules:
                max_score = 0
                best_rule = None
                for s in range(i, j): 
                    for Y, Z in rules[X]:  
                        if not (i, s, Y) in pi or pi[i,s,Y]==None:
                            pi[i,s,Y] = 0
                        elif not (s+1,j,Z) in pi or pi[s+1,j,Z]==None:
                            pi[s+1,j,Z] = 0
                        elif rules_prob[X][(Y, Z)] > 0 and pi[i, s, Y] > 0 and pi[s + 1, j, Z] > 0:
                            score = rules_prob[X][(Y, Z)] * pi[i, s, Y] * pi[s + 1, j, Z]
                            if max_score < score:
                                max_score = score
                                best_rule = Y, Z, s
                bp[i, j, X] = best_rule
                pi[i, j, X] = max_score

    ### re-construction
    max_score = -1
    best_rule = None

    for i,j,X in pi: 
        if i==0 and j== n-1:
            if max_score < pi[0, n-1, X]:
                max_score = pi[0, n-1, X]
                best_rule = 0, n-1, X

    if best_rule == None:
        prediction = "((SENT (NA error)))"
    else:
        prediction = "".join(['((SENT (', get_tree(sentence, bp, *best_rule),')))'])
    return prediction

In [15]:
predicted_sentences = []
precisions = []
NonTerminals = set({a.lhs() for a in grammar.productions()}) # set (dic)
rules, rules_prob = get_rules(grammar)
inversed_lexicon = inverse_lexicon(lexicon)
for i in tqdm.tqdm_notebook(range(len(evaluation))):
    ### get data
    sentence = evaluation_sentences[i].split(' ')
    true = evaluation[i]
    
    ### make prediction
    prediction = cyk(sentence, lexicon, inversed_lexicon, NonTerminals, rules, rules_prob)
    
    ### compute score
    true_tree = parser.create_from_bracket_string(true[1:-1])
    pred_tree = parser.create_from_bracket_string(prediction[1:-1])
    s = scorer.Scorer()
    try:
        result = s.score_trees(true_tree, pred_tree)
        print('Recall =' + str(result.recall))
        print('Precision =' + str(result.prec))
        res_temp = result.recall
    except:
        print('Not correctly parsed.')
        res_temp = 0
    print('\n')
    
    ### store the result
    predicted_sentences.append(prediction)
    precisions.append(res_temp)

Recall =0.9333333333333333
Precision =0.5


Recall =0.7692307692307693
Precision =0.43478260869565216


Recall =0.75
Precision =0.391304347826087


Recall =0.7333333333333333
Precision =0.5238095238095238


Recall =0.4782608695652174
Precision =0.2894736842105263


Not correctly parsed.


Recall =0.7333333333333333
Precision =0.44


Recall =0.4583333333333333
Precision =0.34375


Recall =0.375
Precision =0.2222222222222222


Recall =0.28888888888888886
Precision =0.22033898305084745


Recall =0.30434782608695654
Precision =0.2413793103448276


Recall =0.2553191489361702
Precision =0.15584415584415584


Recall =0.5882352941176471
Precision =0.38461538461538464


Recall =0.4166666666666667
Precision =0.29411764705882354


Recall =0.46153846153846156
Precision =0.35294117647058826


Recall =0.4444444444444444
Precision =0.3333333333333333


Recall =0.4
Precision =0.32


Recall =0.3333333333333333
Precision =0.3333333333333333


Recall =0.6666666666666666
Precision =0.6666666666666666


Re

Recall =0.35714285714285715
Precision =0.29411764705882354


Not correctly parsed.


Not correctly parsed.


Recall =0.5
Precision =0.6666666666666666


Recall =0.2
Precision =0.125


Recall =0.2857142857142857
Precision =0.21428571428571427


Not correctly parsed.


Not correctly parsed.


Recall =0.8
Precision =0.4444444444444444


Recall =0.6666666666666666
Precision =0.4444444444444444


Recall =0.3333333333333333
Precision =0.2222222222222222


Recall =0.3333333333333333
Precision =0.25


Not correctly parsed.


Recall =0.3157894736842105
Precision =0.17142857142857143


Recall =0.2
Precision =0.1111111111111111


Recall =0.3333333333333333
Precision =0.25


Not correctly parsed.


Recall =0.6666666666666666
Precision =0.4


Recall =0.75
Precision =0.6


Recall =0.5
Precision =1.0


Recall =0.5
Precision =0.5


Recall =0.36363636363636365
Precision =0.2553191489361702


Not correctly parsed.


Recall =0.125
Precision =0.1


Not correctly parsed.


Recall =0.6
Precision =0.38709677

In [16]:
### save the prediction
with open('evaluation_data.parser_output.txt', 'w') as f:
    for item in predicted_sentences:
        f.write("%s\n" % item)

In [17]:
np.mean(precisions)

0.33436039691837005

In [19]:
l = np.array(precisions)
np.mean(l[l > 0])

0.4748222988201511

In [20]:
sum([x == 0 for x in precisions])

92